In [1]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D, Conv2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image
from keras.models import model_from_json
import h5py

Using TensorFlow backend.


In [2]:
#path = 'data/sample/'
path = 'data/'

In [3]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
vgg_mean.shape

(3,)

In [4]:
def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.
        Args: 
            x: Image array (height x width x channels)
        Returns:
            Image array (height x width x transposed_channels)
    """
    vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

In [5]:
model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape = (224, 224, 3), output_shape = (224, 224, 3)))

model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))

model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))

model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))

model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))

model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(512, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D((2, 2), strides = (2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation = 'softmax'))

model.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

fname = 'weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
model.load_weights(fname)

In [ ]:
model.summary()

In [6]:
model.pop()
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [7]:
model.add(Dense(2, activation = 'softmax'))
model.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [8]:
batch_size = 52

In [17]:
image.ImageDataGenerator?

In [9]:
# Data Augmentation
trn_datagen = image.ImageDataGenerator(rotation_range=60., zoom_range=0.30, shear_range=0.20,
                                      horizontal_flip=True)
trn_batches = trn_datagen.flow_from_directory(path + 'train/', target_size = (224, 224),
            class_mode = 'categorical', shuffle = True, batch_size = batch_size)

Found 22500 images belonging to 2 classes.


In [12]:
# without Data Augmentation
trn_datagen = image.ImageDataGenerator()
trn_batches = trn_datagen.flow_from_directory(path + 'train/', target_size = (224, 224),
            class_mode = 'categorical', shuffle = True, batch_size = batch_size)

Found 22500 images belonging to 2 classes.


In [13]:
val_datagen = image.ImageDataGenerator()
val_batches = val_datagen.flow_from_directory(path + 'valid/', target_size = (224, 224),
            class_mode = 'categorical', shuffle = True, batch_size = batch_size)

Found 2500 images belonging to 2 classes.


In [11]:
# Data Augmentation
model.fit_generator(trn_batches, steps_per_epoch = trn_batches.n / 52, epochs = 10, validation_data = val_batches, 
                    validation_steps = val_batches.n / 52)

Epoch 1/10
433/432 [==============================] - 648s 1s/step - loss: 0.5608 - acc: 0.8135 - val_loss: 0.2722 - val_acc: 0.9056
Epoch 2/10
433/432 [==============================] - 306s 707ms/step - loss: 0.4735 - acc: 0.8383 - val_loss: 0.2938 - val_acc: 0.8928
Epoch 3/10
433/432 [==============================] - 372s 859ms/step - loss: 0.4709 - acc: 0.8393 - val_loss: 0.2894 - val_acc: 0.9000
Epoch 4/10
433/432 [==============================] - 335s 773ms/step - loss: 0.4933 - acc: 0.8350 - val_loss: 0.3543 - val_acc: 0.8840
Epoch 5/10
433/432 [==============================] - 344s 793ms/step - loss: 0.4647 - acc: 0.8424 - val_loss: 0.2724 - val_acc: 0.9028
Epoch 6/10
433/432 [==============================] - 324s 749ms/step - loss: 0.4910 - acc: 0.8395 - val_loss: 0.2472 - val_acc: 0.9124
Epoch 7/10
433/432 [==============================] - 340s 786ms/step - loss: 0.4740 - acc: 0.8401 - val_loss: 0.2480 - val_acc: 0.9132
Epoch 8/10
433/432 [==============================]

In [14]:
# Without Data Augmentation
model.fit_generator(trn_batches, steps_per_epoch = trn_batches.n / 52, epochs = 10, validation_data = val_batches, 
                    validation_steps = val_batches.n / 52)

Epoch 1/10
433/432 [==============================] - 216s 498ms/step - loss: 0.4357 - acc: 0.8720 - val_loss: 0.2627 - val_acc: 0.9008
Epoch 2/10
433/432 [==============================] - 604s 1s/step - loss: 0.4205 - acc: 0.8742 - val_loss: 0.2337 - val_acc: 0.9136
Epoch 3/10
433/432 [==============================] - 275s 635ms/step - loss: 0.4161 - acc: 0.8729 - val_loss: 0.2333 - val_acc: 0.9200
Epoch 4/10
433/432 [==============================] - 456s 1s/step - loss: 0.4183 - acc: 0.8753 - val_loss: 0.2578 - val_acc: 0.9092
Epoch 5/10
433/432 [==============================] - 343s 792ms/step - loss: 0.4329 - acc: 0.8729 - val_loss: 0.2617 - val_acc: 0.9136
Epoch 6/10
433/432 [==============================] - 222s 513ms/step - loss: 0.4300 - acc: 0.8754 - val_loss: 0.2055 - val_acc: 0.9228
Epoch 7/10
433/432 [==============================] - 218s 504ms/step - loss: 0.4155 - acc: 0.8771 - val_loss: 0.2431 - val_acc: 0.9148
Epoch 8/10
433/432 [==============================] - 